This program will extract all the slices from the 3d image and save as .jpg file for each segment of the nodule.
The result will be saved in the CroppedImageData folder.


In [ ]:
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import zoom
import os
import glob
import matplotlib.image

In [3]:
imgDir = './Data'
dirlist = os.listdir(imgDir)
print(dirlist)
print(len(dirlist))

['LIDC-IDRI-01', 'LIDC-IDRI-02', 'LIDC-IDRI-03', 'LIDC-IDRI-04', 'LIDC-IDRI-05', 'LIDC-IDRI-06', 'LIDC-IDRI-07', 'LIDC-IDRI-08', 'LIDC-IDRI-09', 'LIDC-IDRI-10', 'LIDC-IDRI-11', 'LIDC-IDRI-12', 'LIDC-IDRI-13', 'LIDC-IDRI-14', 'LIDC-IDRI-15', 'LIDC-IDRI-16', 'LIDC-IDRI-17', 'LIDC-IDRI-18', 'LIDC-IDRI-19', 'LIDC-IDRI-20', 'LIDC-IDRI-21', 'LIDC-IDRI-22', 'LIDC-IDRI-23', 'LIDC-IDRI-24', 'LIDC-IDRI-25', 'LIDC-IDRI-26', 'LIDC-IDRI-27', 'LIDC-IDRI-29', 'LIDC-IDRI-30', 'LIDC-IDRI-31', 'LIDC-IDRI-33', 'LIDC-IDRI-34', 'LIDC-IDRI-36', 'LIDC-IDRI-37', 'LIDC-IDRI-39', 'LIDC-IDRI-40', 'LIDC-IDRI-41', 'LIDC-IDRI-42', 'LIDC-IDRI-43', 'LIDC-IDRI-44', 'LIDC-IDRI-45', 'LIDC-IDRI-46', 'LIDC-IDRI-47', 'LIDC-IDRI-48', 'LIDC-IDRI-49', 'LIDC-IDRI-50']
46


In [4]:
#Cropping box
def maskcroppingbox(images_array, use2D=False):
    images_array_2 = np.argwhere(images_array)
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)

In [ ]:
# image segment extraction
for ind in range(len(dirlist)):
    path = os.path.join(imgDir,dirlist[ind])
    imgpath = glob.glob(f'{path}/*.nrrd')
    imgpath = imgpath[0]
    img = sitk.ReadImage(imgpath)

    # pipeline to import data__________________
    # load image
    # load Segmentations for image
    path = os.path.join(path,'Segments')
    pathList = os.listdir(path)

    for segID in range(len(pathList)):  #iterate over every segment
        segPath = os.path.join(imgDir,dirlist[ind],'Segments',pathList[segID])
        mask = sitk.ReadImage(segPath)

        image_array = sitk.GetArrayFromImage(img) 
        mask_array = sitk.GetArrayFromImage(mask)
       
        # creating folder for the segmentation
        newDir = f"{dirlist[ind]}_{str(segID+1)}"
        newDir = os.path.join('CroppedImageData',newDir)
        os.mkdir(newDir)
        (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array, use2D=False)
        roi_images = image_array[zstart-1:zstop+1,ystart:ystop,xstart:xstop].transpose((2,1,0))
        roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1],1], order=3)
        # print(roi_images1.shape)
        roi_images2 = np.array(roi_images1,dtype=float) 
        # print(roi_images2.shape)
               

        transposedImg = roi_images2.transpose((2,1,0))
        for j,i in enumerate(transposedImg):
            matplotlib.image.imsave(f"{newDir}/{j+1}.jpeg",i, cmap='gray')
    print(dirlist[ind])
